In [1]:
## packages
import pandas as pd
import numpy as np
import tarfile
import json

## unzip tar
# import tarfile
# tar = tarfile.open('yelp_dataset.tarf]=', 'r:gz')
# tar.extractall()
# tar.close()

In [2]:
# reviews = pd.read_json('yelp_academic_dataset_review.json', lines = True)
# users = pd.read_json('yelp_academic_dataset_user.json', lines = True)
# businesses = pd.read_json('yelp_academic_dataset_business.json', lines = True)
philly_bus = pd.read_feather('../FilteredData/business_philly.feather')
philly_reviews = pd.read_feather('../FilteredData/review_philly.feather')
philly_users = pd.read_feather('../FilteredData/user_philly.feather')

In [3]:
# business_cat = businesses.categories.unique
# df = businesses[businesses['categories'].str.contains('Restaurant')]
philly_reviews

,_id,review_id,user_id,business_id,stars,useful,funny,cool,text,date,compliment_count
0,631e9f7fedf65856ab0dfe3b,JrIxlS1TzJ-iCu79ul40cQ,eUta8W_HdHMXPzLBBZhL1A,04UD14gamNjLY0IDYVhHJg,1.0,1.0,2.0,1.0,I am a long term frequent customer of this est...,2015-09-23 23:10:31,NaN
1,631e9f7fedf65856ab0dfe3e,8JFGBuHMoiNDyfcxuWNtrA,smOvOajNG0lS4Pq7d8g4JQ,RZtGWDLCAtuipwaZ-UfjmQ,4.0,0.0,0.0,0.0,Good food--loved the gnocchi with marinara\nth...,2009-10-14 19:57:14,NaN
2,631e9f7fedf65856ab0dfe42,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,1.0,0.0,1.0,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03,NaN
3,631e9f7fedf65856ab0dfe43,J-4NdnDZ0pUQaUEEwDI9KQ,vrKkXsozqqecF3CW4cGaVQ,rjuWz_AD3WfXJc03AhIO_w,5.0,2.0,2.0,2.0,I thoroughly enjoyed the show. Chill way to s...,2012-12-04 16:46:20,NaN
4,631e9f7fedf65856ab0dfe55,JBWZmBy69VMggxj3eYn17Q,aFa96pz67TwOFu4Weq5Agg,kq5Ghhh14r-eCxlVmlyd8w,5.0,0.0,0.0,0.0,My boyfriend and I tried this deli for the fir...,2018-08-23 21:39:38,NaN
...,...,...,...,...,...,...,...,...,...,...,...
855075,6327b03050426527167a1746,nan,uChvFHK4uSifUXBPJPp9Zw,lCetcbxQr52TTGmGmXzQcw,NaN,NaN,NaN,NaN,Racist!!! Wont serve you if you are ethnic,2013-12-15 04:51:26,1.0
855076,6327b03050426527167a1749,nan,lP_cx94U8stOuQ8HrGecdQ,03jQGGJ2ch0uHTtW-UUUqg,NaN,NaN,NaN,NaN,Stuff face time.,2012-05-27 14:06:04,0.0
855077,6327b03050426527167a174c,nan,Rr4cLb6Go91FT134o6RsKg,UEGZ1nHUVb2ltGtd7C7X9w,NaN,NaN,NaN,NaN,Blizzard,2012-06-05 20:34:27,0.0
855078,6327b03050426527167a175b,nan,pflneuKFKQKQvZGDxebGFw,zsaDvomuMLe_-Ibtk9sA-A,NaN,NaN,NaN,NaN,Stopped by and it was closed on wed. 10/3/12!,2012-10-03 16:33:53,0.0


In [33]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k,auc_score,reciprocal_rank
from lightfm.data import Dataset
from lightfm import LightFM, cross_validation

In [48]:
# #split test and train data based on reviews
# reviews_only['train'] = np.random.uniform(0, 1, len(reviews_only)) <= .75
# reviews_train = reviews_only[["user_id", "business_id", "stars"]][reviews_only.train==True]
# reviews_test = reviews_only[reviews_only.train==False]

# users_filtered = users[~users.user_id.isin(reviews.user_id)]
df = philly_reviews.groupby('user_id')['stars'].mean()
users = pd.merge(philly_users, df, on=['user_id'], how='left')

In [35]:
#unique user features
uf = []
col = ['stars']*len(users['stars'].unique()) 
unique_list = list(users['stars'].unique())
# col = ['review_count']*len(users['review_count'].unique()) + ['useful']*len(users['useful'].unique()) + ['funny']*len(users['funny'].unique()) + ['cool']*len(users['cool'].unique())
# unique_list = list(users['review_count'].unique()) + list(users['useful'].unique()) + list(users['funny'].unique()) + list(users['cool'].unique())


for x,y in zip(col, unique_list):
    res = str(x)+ ":" +str(y)
    uf.append(res)
#     print(res)

In [6]:
# list(filter(lambda x: '27281' in x, uf))

In [36]:
reviews_only = philly_reviews[["user_id", "business_id", "stars"]]

In [37]:
dataset1 = Dataset()
dataset1.fit(
        philly_reviews['user_id'].unique(), # all the users
        philly_reviews['business_id'].unique(), # all the items
        user_features = uf # additional user features
)

In [39]:
def feature_colon_value(my_list):
    """
    Takes as input a list and prepends the columns names to respective values in the list.
    For example: if my_list = [1,1,0,'del'],
    resultant output = ['f1:1', 'f2:1', 'f3:0', 'loc:del']
   
    """
    result = []
    ll = ['stars:']
    # ll = ['review_count:', 'useful:', 'funny:', 'cool:']
    aa = my_list
    for x,y in zip(ll,aa):
        res = str(x) +""+ str(y)
        result.append(res)
    return result
# Using the helper function to generate user features in proper format for ALL users
ad_subset = users[['stars']] 
# ad_subset = philly_users[["review_count",'useful', 'funny', 'cool']] 
ad_list = [list(x) for x in ad_subset.values]
feature_list = []
for item in ad_list:
    feature_list.append(feature_colon_value(item))
# print(f'Final output: {feature_list}')


In [40]:
user_tuple = list(zip(philly_users.user_id, feature_list))

In [41]:
# user_tuple
user_features = dataset1.build_user_features(user_tuple, normalize= False)

In [42]:
(interactions, weights) = dataset1.build_interactions([(x[0], x[1], x[2]) for x in reviews_only.values])

In [43]:
train, test = cross_validation.random_train_test_split(interactions, test_percentage=0.25)

In [44]:
model = LightFM() #loss='warp'
model.fit(train,
      user_features= user_features,
      # item_features = item_features
      epochs=30,
      num_threads=2)

In [45]:
from lightfm.evaluation import auc_score
train_auc = auc_score(model,
                      train,
                      user_features=user_features
                     ).mean()
print('Hybrid training set AUC: %s' % train_auc)

Hybrid training set AUC: 0.82521945
